In [56]:
!pip install pandas numpy matplotlib seaborn
!pip install pulp
!pip install glpk
!pip install python-dateutil



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [57]:
# 数据处理：从原始Anon Enrollment Data_new.xlsx中提取数院学生选课情况的数据，并输出为math_student_enrollment.xlsx
# 每个学生和每个课都有其对应的index，若学生s选择了i，则Y_si=1。(s和i都是index)
import pandas as pd
import numpy as np

def process_math_student_enrollment(file_path):
    """
    处理数学院学生的选课数据（仅处理'Course Enrollments'工作表）
    
    参数:
        file_path (str): 选课数据Excel文件的路径
        
    返回:
        pandas.DataFrame: 数学院学生选课数据
        dict: 学生ID到索引的映射
        dict: 课程ID到索引的映射
        dict: 学生-课程映射
    """
    print(f"正在处理数学院学生选课数据: {file_path}")
    
    try:
        # 仅读取'Course Enrollments'工作表
        df = pd.read_excel(file_path, sheet_name='Course Enrollments')
        
        # 打印原始数据的基本信息
        print(f"原始数据形状: {df.shape}")
        print(f"原始数据列: {df.columns.tolist()}")
        
        # 筛选数学院的学生数据和MATH开头的课程
        math_students = df[
            (df['Programme School Name'] == 'School of Mathematics') & 
            (df['Course Code'].str.startswith('MATH'))
        ].copy()
        print(f"数学院学生MATH课程数据形状: {math_students.shape}")
        
        # 提取所需的列: UNN、Course Code、Programme Of Study Sought Title等
        enrollment_data = math_students[['UNN', 'Course Code', 'Programme Of Study Sought Title']].copy()
        
        # 重命名列为标准格式
        enrollment_data = enrollment_data.rename(columns={
            'UNN': 'student_id',
            'Course Code': 'course_id',
            'Programme Of Study Sought Title': 'programme'
        })
        
        # 确保没有重复记录
        enrollment_data = enrollment_data.drop_duplicates()
        
        # 创建学生ID到索引的映射
        student_id_to_index = {student_id: i+1 for i, student_id in enumerate(enrollment_data['student_id'].unique())}
        
        # 创建课程ID到索引的映射
        course_id_to_index = {course_id: i+1 for i, course_id in enumerate(enrollment_data['course_id'].unique())}
        
        # 添加索引列
        enrollment_data['student_index'] = enrollment_data['student_id'].map(student_id_to_index)
        enrollment_data['course_index'] = enrollment_data['course_id'].map(course_id_to_index)
        
        print(f"处理后的数据形状: {enrollment_data.shape}")
        print(f"数学院学生人数: {len(student_id_to_index)}")
        print(f"MATH课程数量: {len(course_id_to_index)}")
        
        # 将结果保存为Excel文件，以便于后续使用
        output_file = 'math_student_enrollment.xlsx'
        enrollment_data.to_excel(output_file, index=False)
        print(f"数学院学生MATH课程选课数据已保存到: {output_file}")
        
        # 创建学生-课程映射
        student_courses = {}
        for _, row in enrollment_data.iterrows():
            student_idx = row['student_index']
            course_idx = row['course_index']
            if student_idx not in student_courses:
                student_courses[student_idx] = []
            student_courses[student_idx].append(course_idx)
        
        return enrollment_data, student_id_to_index, course_id_to_index, student_courses
        
    except Exception as e:
        print(f"处理数学院学生选课数据时出错: {str(e)}")
        return None, None, None, None

# 示例用法
if __name__ == "__main__":
    file_path = "/Users/ashley/Desktop/Topics in Applied OR/code_qp/topics/Anon Enrollment Data_new.xlsx"
    enrollment_data, student_id_to_index, course_id_to_index, student_courses = process_math_student_enrollment(file_path)
    
    if enrollment_data is not None:
        # 打印部分学生选课信息作为示例
        print("\n学生选课示例:")
        for student_idx, courses in list(student_courses.items())[:5]:  # 显示前5个学生的选课
            # 找回学生ID
            student_id = [sid for sid, idx in student_id_to_index.items() if idx == student_idx][0]
            # 找回课程ID
            course_ids = [cid for cid, idx in course_id_to_index.items() if idx in courses]
            print(f"学生 {student_id} 选了以下MATH课程: {course_ids}")

正在处理数学院学生选课数据: /Users/ashley/Desktop/Topics in Applied OR/code_qp/topics/Anon Enrollment Data_new.xlsx
原始数据形状: (14694, 7)
原始数据列: ['UNN', 'he', 'Programme Of Study Sought Title', 'Programme School Name', 'Normal Year Taken', 'Course Code', 'Course Name']
数学院学生MATH课程数据形状: (9164, 7)
处理后的数据形状: (9163, 5)
数学院学生人数: 1317
MATH课程数量: 129
数学院学生MATH课程选课数据已保存到: math_student_enrollment.xlsx

学生选课示例:
学生 V2173617 选了以下MATH课程: ['MATH08062', 'MATH08071', 'MATH08065', 'MATH08064', 'MATH08066', 'MATH08063', 'MATH08051']
学生 Q7802173 选了以下MATH课程: ['MATH08062', 'MATH08071', 'MATH08065', 'MATH08064', 'MATH08066', 'MATH08063', 'MATH08051']
学生 J5970413 选了以下MATH课程: ['MATH08062', 'MATH08071', 'MATH08066', 'MATH08063']
学生 C7307748 选了以下MATH课程: ['MATH08062', 'MATH08071', 'MATH08065', 'MATH08064', 'MATH08066']
学生 M2864203 选了以下MATH课程: ['MATH08062', 'MATH08071', 'MATH08065', 'MATH08064', 'MATH08066', 'MATH08063', 'MATH08051']


In [58]:
import pandas as pd
import numpy as np
import time
import re
from ortools.sat.python import cp_model
import matplotlib.pyplot as plt
import logging

# 配置日志记录
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [59]:
# 数据处理：处理教学周的问题
def preprocess_course_data(courses_df):
    """
    预处理课程数据
    
    参数:
    courses_df (pandas.DataFrame): 原始课程数据
    
    返回:
    pandas.DataFrame: 预处理后的课程数据
    """
    # 解析教学周模式
    def parse_week_pattern(pattern):
        if not isinstance(pattern, str):
            return []
        
        weeks = []
        parts = pattern.split(',')
        
        for part in parts:
            part = part.strip()
            if '-' in part:
                start, end = map(int, part.split('-'))
                weeks.extend(range(start, end + 1))
            else:
                try:
                    weeks.append(int(part))
                except:
                    pass
        
        return sorted(weeks)
    
    # 确定周期性
    def determine_week_parity(delivery_semester):
        if not isinstance(delivery_semester, str):
            return 'ALL'
        
        lower_str = delivery_semester.lower()
        if 'odd' in lower_str or 'uneven' in lower_str:
            return 'ODD'
        elif 'even' in lower_str:
            return 'EVEN'
        else:
            return 'ALL'
    
    # 添加解析后的教学周列表
    courses_df['Teaching_Weeks'] = courses_df['Teaching Week Pattern'].apply(parse_week_pattern)
    courses_df['Week_Parity'] = courses_df['Delivery Semester'].apply(determine_week_parity)
    
    # 应用单双周筛选到教学周
    for idx, row in courses_df.iterrows():
        if row['Week_Parity'] == 'ODD':
            courses_df.at[idx, 'Teaching_Weeks'] = [w for w in row['Teaching_Weeks'] if w % 2 == 1]
        elif row['Week_Parity'] == 'EVEN':
            courses_df.at[idx, 'Teaching_Weeks'] = [w for w in row['Teaching_Weeks'] if w % 2 == 0]
    
    return courses_df

def identify_workshop_merge_opportunities(courses_df):
    """
    识别潜在的workshop合并机会，特别关注 Real Size 的处理
    
    参数:
    courses_df (pandas.DataFrame): 预处理后的课程数据
    
    返回:
    list: 潜在合并的workshop组
    """
    # 创建时间标识
    courses_df['Day_Time'] = courses_df['Scheduled Days'] + '_' + courses_df['Scheduled Start Time']
    
    potential_merges = []
    course_codes = courses_df['Course Code'].unique()
    
    for course_code in course_codes:
        course_subset = courses_df[courses_df['Course Code'] == course_code]
        
        day_times = course_subset['Day_Time'].unique()
        for day_time in day_times:
            workshops = course_subset[course_subset['Day_Time'] == day_time]
            if len(workshops) > 1:
                week_patterns = workshops['Teaching Week Pattern'].unique()
                parities = workshops['Week_Parity'].unique()
                
                if len(week_patterns) == 1 and len(parities) == 1:
                    potential_merges.append({
                        'course_code': course_code,
                        'day_time': day_time,
                        'workshop_indices': workshops.index.tolist(),
                        'total_size': workshops['Real Size'].sum(),  # 使用 Real Size 求和
                        'merged_name': f"{course_code}_{day_time}_MERGED",
                        'week_pattern': week_patterns[0],
                        'parity': parities[0],
                        'teaching_weeks': workshops.iloc[0]['Teaching_Weeks']
                    })
    
    return potential_merges

def convert_weekday_to_week_and_day(weekday_index):
    """
    根据数学模型的weekday索引逻辑转换
    例如: 
    11 -> (2, 1)  # 第2周的第1天
    12 -> (2, 2)  # 第2周的第2天
    """
    week = (weekday_index - 1) // 5 + 1
    day_in_week = (weekday_index - 1) % 5 + 1
    return week, day_in_week

In [60]:
# 数据处理：处理slot对应具体哪周哪天的问题
def convert_day_to_index(day_str):
    """将星期几字符串转换为索引(0-4)"""
    if not isinstance(day_str, str):
        return None
    
    day_str = day_str.lower()
    if 'mon' in day_str:
        return 0
    elif 'tue' in day_str:
        return 1
    elif 'wed' in day_str:
        return 2
    elif 'thu' in day_str:
        return 3
    elif 'fri' in day_str:
        return 4
    return None

def convert_time_to_slot(time_str):
    """将时间字符串(如'09:00')转换为时间段索引(1-18)"""
    if not isinstance(time_str, str):
        return None
    
    # 尝试解析"HH:MM"格式
    match = re.match(r'(\d{1,2}):(\d{2})', time_str)
    if match:
        hour = int(match.group(1))
        minute = int(match.group(2))
        
        # 计算时间段索引 (9:00为1, 9:30为2, 以此类推)
        if 9 <= hour < 18:
            slot = (hour - 9) * 2 + 1
            if minute >= 30:
                slot += 1
            return slot
    
    return None

def convert_slot_to_time(time_slot):
    """将时间段索引(1-18)转换为时间字符串(如'09:00')"""
    if not isinstance(time_slot, int) or time_slot < 1 or time_slot > 18:
        return None
    
    hour = 9 + ((time_slot - 1) // 2)
    minute = 30 if (time_slot - 1) % 2 == 1 else 0
    return f"{hour:02d}:{minute:02d}"

In [ ]:
def multi_objective_course_scheduling(
    enrollment_file='math_student_enrollment.xlsx', 
    courses_file='df_final_cleaned_1.xlsx', 
    rooms_file='Timetabling_KB_Rooms.xlsx'
):
    """
    多目标课程排课优化模型
    
    模型集合定义:
    - I: 课程集合
    - J: 185个weekdays (37周 * 5天)
    - K: 教室集合
    - T: 18个时间槽
    - S: 学生集合
    
    目标函数:
    f1: 最大化教室利用率
    f2: 最小化学生课程冲突
    f3: 最小化课程分布方差
    """
    # 1. 数据加载与预处理
    try:
        # 加载数据
        courses_df = pd.read_excel(courses_file)
        rooms_df = pd.read_excel(rooms_file)
        enrollment_df = pd.read_excel(enrollment_file)
        
        # 预处理课程数据
        courses_df = preprocess_course_data(courses_df)
        
        print(f"课程数量: {len(courses_df)}")
        print(f"教室数量: {len(rooms_df)}")
        print(f"学生选课记录: {len(enrollment_df)}")
    except Exception as e:
        print(f"数据加载失败: {str(e)}")
        return None
    
    # 2. 创建优化模型
    model = cp_model.CpModel()
    
    # 3. 定义决策变量 Xijkt
    # X[i,j,k,t] = 1 表示课程i在第j个weekday的第k个教室t时间槽被安排
    X = {}
    
    # 4. 准备数据映射
    # 课程ID到索引的映射
    course_to_index = {course: idx+1 for idx, course in enumerate(courses_df['Course Code'])}
    
    # 教室ID到索引的映射
    room_to_index = {room: idx+1 for idx, room in enumerate(rooms_df['ROOM NAME'])}
    
    # 5. 创建决策变量
    for i, course in courses_df.iterrows():
        course_code = course['Course Code']
        course_idx = course_to_index[course_code]
        
        # 考虑教学周
        for j in range(1, 186):  # weekdays
            week, day_in_week = convert_weekday_to_week_and_day(j)
            
            # 检查是否符合教学周要求
            if week not in course['Teaching_Weeks']:
                continue
            
            for k, room in rooms_df.iterrows():
                room_idx = room_to_index[room['ROOM NAME']]
                
                for t in range(1, 19):  # 时间槽
                    # 创建决策变量
                    X[course_idx, j, room_idx, t] = model.NewBoolVar(
                        f'X_{course_idx}_{j}_{room_idx}_{t}'
                    )
    
    # 6. 约束条件
    
    # 约束1: 一个教室同一时间只能安排一门课程
    for j in range(1, 186):
        for room_idx in range(1, len(rooms_df)+1):
            for t in range(1, 19):
                model.Add(
                    sum(X[course_idx, j, room_idx, t] 
                        for course_idx in range(1, len(courses_df)+1) 
                        if (course_idx, j, room_idx, t) in X) <= 1
                )
    
    # 约束2: 一门课程同一时间不能在多个教室
    for course_idx in range(1, len(courses_df)+1):
        for j in range(1, 186):
            model.Add(
                sum(X[course_idx, j, room_idx, t] 
                    for room_idx in range(1, len(rooms_df)+1)
                    for t in range(1, 19)
                    if (course_idx, j, room_idx, t) in X) <= 1
            )
    
    # 约束3: 教室容量约束
    for i, course in courses_df.iterrows():
        course_idx = course_to_index[course['Course Code']]
        course_real_size = course['Real Size']
        
        for j in range(1, 186):
            for k, room in rooms_df.iterrows():
                room_idx = room_to_index[room['ROOM NAME']]
                room_capacity = room['CAP']
                
                for t in range(1, 19):
                    if (course_idx, j, room_idx, t) in X:
                        # 使用大M方法
                        model.Add(
                            course_real_size <= room_capacity + (1 - X[course_idx, j, room_idx, t]) * 10000
                        )
    
    # 约束5: 每周最多安排一次课
    for course_idx in range(1, len(courses_df)+1):
        for week in range(1, 38):  # 37周
            week_start = 5 * (week - 1) + 1
            week_end = 5 * week
            
            model.Add(
                sum(X[course_idx, j, room_idx, t] 
                    for j in range(week_start, week_end + 1)
                    for room_idx in range(1, len(rooms_df)+1)
                    for t in range(1, 19)
                    if (course_idx, j, room_idx, t) in X) <= 1
            )
    
    # 7. 目标函数
    # f1: 最大化教室利用率（最小化未充分利用的空间）
    f1_terms = []
    for i, course in courses_df.iterrows():
        course_idx = course_to_index[course['Course Code']]
        course_real_size = course['Real Size']
        
        for j in range(1, 186):
            for k, room in rooms_df.iterrows():
                room_idx = room_to_index[room['ROOM NAME']]
                room_capacity = room['CAP']
                
                for t in range(1, 19):
                    if (course_idx, j, room_idx, t) in X:
                        # 创建一个新的辅助变量来表示利用率
                        utilization_var = model.NewIntVar(0, 10000, f'utilization_{course_idx}_{j}_{room_idx}_{t}')
                        
                        # 定义利用率计算
                        model.Add(utilization_var == 
                                  X[course_idx, j, room_idx, t] * course_real_size * 100 // room_capacity)
                        
                        # 添加到目标函数项
                        f1_terms.append(-utilization_var)
    
    # f2: 最小化学生课程冲突
    f2_terms = []
    for j in range(1, 186):  # weekdays
        for t in range(1, 19):  # 时间槽
            for student_id, student_row in enrollment_df.iterrows():
                # 学生选的课程
                student_courses = [
                    course_to_index[course] 
                    for course in student_row['Course Code'].split(',')
                ]
                
                # 检查学生在该时间点是否有多门课
                courses_in_slot = [
                    X[course_idx, j, room_idx, t]
                    for course_idx in student_courses
                    for room_idx in range(1, len(rooms_df)+1)
                    if (course_idx, j, room_idx, t) in X
                ]
                
                if len(courses_in_slot) > 1:
                    conflict_var = model.NewBoolVar(f'conflict_{j}_{t}_{student_id}')
                    model.Add(sum(courses_in_slot) > 1).OnlyEnforceIf(conflict_var)
                    model.Add(sum(courses_in_slot) <= 1).OnlyEnforceIf(conflict_var.Not())
                    f2_terms.append(conflict_var)
    
    # f3: 最小化课程分布方差
    f3_terms = []
    for t in range(1, 19):  # 时间槽
        courses_in_slot = [
            X[course_idx, j, room_idx, t]
            for course_idx in range(1, len(courses_df)+1)
            for j in range(1, 186)
            for room_idx in range(1, len(rooms_df)+1)
            if (course_idx, j, room_idx, t) in X
        ]
        f3_terms.append(sum(courses_in_slot))
    
    # 权重
    w1, w2, w3 = 1, 10, 5
    
    # 组合目标函数
    model.Minimize(
        w1 * sum(f1_terms) + 
        w2 * sum(f2_terms) + 
        w3 * np.var(f3_terms)
    )
    
    # 8. 求解模型
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 600  # 10分钟求解时间
    
    status = solver.Solve(model)
    
    # 9. 处理结果
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        # 提取排课结果
        schedule = []
        for i, course in courses_df.iterrows():
            course_idx = course_to_index[course['Course Code']]
            
            for j in range(1, 186):
                for k, room in rooms_df.iterrows():
                    room_idx = room_to_index[room['ROOM NAME']]
                    
                    for t in range(1, 19):
                        if (course_idx, j, room_idx, t) in X and solver.Value(X[course_idx, j, room_idx, t]) == 1:
                            schedule.append({
                                'Course Code': course['Course Code'],
                                'Weekday Index': j,
                                'Room': room['ROOM NAME'],
                                'Time Slot': t
                            })
        
        # 保存结果
        schedule_df = pd.DataFrame(schedule)
        schedule_df.to_excel('multi_objective_course_schedule.xlsx', index=False)
        
        print(f"排课完成，共安排 {len(schedule)} 个课程时段")
        print("结果已保存到 multi_objective_course_schedule.xlsx")
        
        return schedule_df
    else:
        print("无法找到可行的排课方案")
        return None

In [64]:
# 调用排课函数
result = multi_objective_course_scheduling(
    enrollment_file='math_student_enrollment.xlsx',
    courses_file='df_final_cleaned_1.xlsx',
    rooms_file='Timetabling_KB_Rooms.xlsx'
)

# 处理结果
if result is not None:
    # 打印前几行排课结果
    print(result.head())
    
    # 如果需要，可以进一步分析结果
    print("\n排课统计信息:")
    print(f"总排课课程数: {len(result)}")
    
    # 可以按课程分组查看排课情况
    course_distribution = result.groupby('Course Code').size()
    print("\n各课程排课数量:")
    print(course_distribution)

课程数量: 557
教室数量: 58
学生选课记录: 9163


NotImplementedError: calling // on a linear expression is not supported, please use CpModel.add_division_equality